## Import funcions

In [2]:
#!conda install --file requirements.txt
import os
from agents.functions import *
from agents.agent_class import *
from os.path import join
from collections import defaultdict
import pandas as pd
import re
import csv
from datetime import date
import argparse
from numpy.random import choice, shuffle

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
from agents.extract_belif import *
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

### OpenAi API key

In [3]:
if os.getenv("GOOGLE_API_KEY"):
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
else:
    with open("api/googleai-key.txt", "r") as f:
        GOOGLE_API_KEY = f.read().strip()

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
reflection_count = 0


if os.getenv("OPENAI_API_KEY"):
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
else:
    with open("api/openai-key.txt", "r") as f:
        OPENAI_API_KEY = f.read().strip()

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
reflection_count = 0


### Argument parser

In [6]:
import sys
from argparse import ArgumentParser

# Simula gli argomenti della riga di comando
sys.argv = [
    "notebook",
    "--model_name", "gemini-pro",# gemini-pro
    "--temperature", "0.7",
    "--num_agents", "3",
    "--num_steps", "3",
    "--distribution", "uniform",
    "--version_set", "",
    "--seed", "1",
    "--output_file", "resume.csv",
    "--output_path", "output"
]

# Configura il parser
parser = ArgumentParser(description="Argument Parser for Opinion Dynamics Script")
parser.add_argument(
    "-m",
    "--model_name",
    default="gpt-3.5-turbo-16k",
    type=str,
    help="Name of the LLM to use as agents",
)
parser.add_argument(
    "-t",
    "--temperature",
    default=0.7,
    type=float,
    help="Parameter that influences the randomness of the model's responses",
)
parser.add_argument(
    "-agents",
    "--num_agents",
    default=2,
    type=int,
    help="Number of agents participating in the study",
)
parser.add_argument(
    "-steps",
    "--num_steps",
    default=100,
    type=int,
    help="Number of steps or pair samples in the experiment",
)
parser.add_argument(
    "-dist",
    "--distribution",
    default="uniform",
    choices=["uniform", "skewed_positive", "skewed_negative", "positive", "negative"],
    type=str,
    help="Type of initial opinion distribution",
)
parser.add_argument(
    "-version",
    "--version_set",
    default="v61_control",
    type=str,
    help="Prompt version directory to use",
)
parser.add_argument(
    "-seed",
    "--seed",
    default=1,
    type=int,
    help="Set reproducibility seed",
)
parser.add_argument("-test", "--test_run", action="store_true", help="Set flag if test run")
parser.add_argument("--no_rating", action="store_true", help="Set flag if prompt is no rating")
parser.add_argument(
    "-out", "--output_file", type=str, default="test1.csv", help="Name of the output file"
)
parser.add_argument( "--output_path", type=str, default="output", help="Path of the output file")

# Analizza gli argomenti
args = parser.parse_args()

# Usa gli argomenti nel tuo script
print(f"Model Name: {args.model_name}")
print(f"Temperature: {args.temperature}")
print(f"Number of Agents: {args.num_agents}")
print(f"Number of Steps: {args.num_steps}")
print(f"Distribution: {args.distribution}")
print(f"Version Set: {args.version_set}")
print(f"Output File: {args.output_file}")
print(f"Test Run: {args.test_run}")
print(f"No Rating: {args.no_rating}")
print(f"Output Path: {args.output_path}")


Model Name: gemini-pro
Temperature: 0.7
Number of Agents: 3
Number of Steps: 3
Distribution: uniform
Version Set: 
Output File: resume.csv
Test Run: False
No Rating: False
Output Path: output


## Main

In [7]:
# Esempio di utilizzo
if __name__ == "__main__":
    df_agents = pd.read_csv(join('prompts', "list_agent_descriptions.csv"))
    df_agents_selected=df_agents.sample(n=args.num_agents, random_state=args.seed)


    list_agents = []
    list_agent_ids = df_agents_selected["agent_id"]
    list_agent_persona = df_agents_selected["persona"]
    for persona, agent_id in zip(list_agent_persona, list_agent_ids):
        agent = Agent(agent_id=agent_id,persona=persona, 
                      model_name=args.model_name,temperature=args.temperature,max_tokens=100,prompt_template_root="prompts")
        # And add them to the list of agents
        list_agents.append(agent)
        print(f"Agente {agent.agent_name} creato con opinione iniziale: {agent.init_belief}")



Agente Logan Smith creato con opinione iniziale: -1
Agente Daniel Harris creato con opinione iniziale: -2
Agente Linda Evans creato con opinione iniziale: 0


### Interactions

In [8]:
from collections import defaultdict
from tqdm import tqdm
from datetime import datetime

timestamp =datetime.now().strftime('%Y%m%d%H%M%S')
output_dir = os.path.join(args.output_path, str(timestamp))
os.makedirs(output_dir, exist_ok=True)

dict_agent_tweet = defaultdict(list)
dict_agent_response = defaultdict(list)
dict_csv = dict()
for agent in list_agents:
    dict_csv["time_step"] = [0]
    dict_csv[agent.agent_name] = [agent.init_belief]

interaction_out_name = join(args.output_path,str(timestamp),args.output_file)
with open(interaction_out_name, "w+") as f:
    writer = csv.writer(f, delimiter=",")
    writer.writerow(
        [
            "Time Step",
            "Agent_J Name",
            "Agent_J Belief",
            "Agent_J Tweet",
            "Agent_I Name",
            "Agent_I Pre-Belief",
            "Agent_I Response",
            "Agent_I Post-Belief",
            "Agent_I Delta-Belief",
        ]
    )
    for t in tqdm(range(args.num_steps)):
        dict_csv["time_step"].append(t + 1)
        row = []
        row.append(t + 1)

        # Get a random pair of agents
        agent_i, agent_j = get_random_pair(list_agents)
        row.append(agent_j.agent_name)

        # If the agent is no longer "fresh", outdate the memoery about their persona and the instruction
        if agent_i.get_count_tweet_seen() + agent_i.get_count_tweet_written() == 1:
            agent_i.outdate_persona_memory()
        if agent_j.get_count_tweet_seen() + agent_j.get_count_tweet_written() == 1:
            agent_j.outdate_persona_memory()

        row.append(agent_j.current_belief)
        agent_j_previos_interaction_type = agent_j.previous_interaction_type
        if agent_j_previos_interaction_type in ["none", "write", "read"]:
            # Agent j writes a tweet
            tweet_j = agent_j.produce_tweet(previous_interaction_type=agent_j_previos_interaction_type,tweet_written_count=agent_j.get_count_tweet_written(),add_to_memory=False,)
            agent_j.increase_count_tweet_written()
            print(tweet_j)
            row.append(tweet_j)
        else:
            raise ValueError(
                "agent_j_previos_interaction_type is not valid: {}".format(
                    agent_j_previos_interaction_type
                )
            )
        # Add to Agent j's memory
        agent_j.add_to_memory(
            tweet_written=tweet_j,
            previos_interaction_type=agent_j_previos_interaction_type,
            current_interaction_type="write",
            tweet_written_count=agent_j.get_count_tweet_written(),
        )
        # Agent i appears and reads the tweet
        row.append(agent_i.agent_name)
        agent_i_pre_belief = agent_i.current_belief
        row.append(agent_i_pre_belief)
        agent_i_previos_interaction_type = agent_i.previous_interaction_type
        if agent_i_previos_interaction_type in ["none", "write", "read"]:
            response_i = agent_i.receive_tweet(
                tweet_j,
                previous_interaction_type=agent_i_previos_interaction_type,
                tweet_written_count=agent_i.get_count_tweet_written(),
                add_to_memory=False,
            )
            agent_i.increase_count_tweet_seen()
            row.append(response_i)
        else:
            raise ValueError(
                "agent_i_previos_interaction_type is not valid: {}".format(
                    agent_i_previos_interaction_type
                )
            )
        # Add to Agent i's memory
        agent_i.add_to_memory(
                tweet_seen=tweet_j,
                response=response_i,
                previos_interaction_type=agent_i_previos_interaction_type,
                current_interaction_type="read",
                tweet_written_count=agent_i.get_count_tweet_written(),
            )
        agent_i.current_belief = extract_belief(response_i)
        agent_i_post_belief = agent_i.current_belief
        row.append(agent_i_post_belief)
        row.append(agent_i_post_belief - agent_i_pre_belief if pd.notna(agent_i_post_belief) and pd.notna(agent_i_pre_belief) else 'NA')

        writer.writerow(row)

        agent_i.previous_interaction_type = "read"
        agent_j.previous_interaction_type = "write"

        for agent in list_agents:
            dict_csv[agent.agent_name].append(agent.current_belief)

        dict_agent_tweet[agent_j].append((t + 1, agent_j.current_belief, tweet_j))
        dict_agent_response[agent_i].append((t + 1, response_i))
pd.DataFrame.from_dict(dict_csv).to_csv(join(args.output_path,str(timestamp), 'opinion_ts.csv'), index=False)
    


  0%|          | 0/3 [00:00<?, ?it/s]



> Entering new ConversationChain chain...
Prompt after formatting:
System: Role play this person.

belief: Slightly Negative, name: Logan Smith,political_leaning: Lean Republican, age: 39, gender: Male, ethnicity: African American, education: Trade School Graduate (Electrician), occupation: Electrician, background: Logan is slightly skeptical of the VAX due to concerns about its quick rollout. He values personal health and freedom of choice, leaning him slightly towards a negative perspective on the vaccine.

Now, Logan Smith, you have been interacting with other strangers on Twitter. You can decide to change or maintain your belief about the Theory XYZ that claims that on a clear sunny day, the sky is usually red. after interacting with other strangers.

You would produce tweets that reflect your honest belief, and you would also see other stangers’ tweets. After seeing other people’s tweets, you would be asked about your belief about the Theory XYZ that claims that on a clear sunny

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..




> Entering new ConversationChain chain...
Prompt after formatting:
System: Role play this person.

belief: Slightly Negative, name: Logan Smith,political_leaning: Lean Republican, age: 39, gender: Male, ethnicity: African American, education: Trade School Graduate (Electrician), occupation: Electrician, background: Logan is slightly skeptical of the VAX due to concerns about its quick rollout. He values personal health and freedom of choice, leaning him slightly towards a negative perspective on the vaccine.

Now, Logan Smith, you have been interacting with other strangers on Twitter. You can decide to change or maintain your belief about the Theory XYZ that claims that on a clear sunny day, the sky is usually red. after interacting with other strangers.

You would produce tweets that reflect your honest belief, and you would also see other stangers’ tweets. After seeing other people’s tweets, you would be asked about your belief about the Theory XYZ that claims that on a clear sunny

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..




> Entering new ConversationChain chain...
Prompt after formatting:
System: Role play this person.

belief: Slightly Negative, name: Logan Smith,political_leaning: Lean Republican, age: 39, gender: Male, ethnicity: African American, education: Trade School Graduate (Electrician), occupation: Electrician, background: Logan is slightly skeptical of the VAX due to concerns about its quick rollout. He values personal health and freedom of choice, leaning him slightly towards a negative perspective on the vaccine.

Now, Logan Smith, you have been interacting with other strangers on Twitter. You can decide to change or maintain your belief about the Theory XYZ that claims that on a clear sunny day, the sky is usually red. after interacting with other strangers.

You would produce tweets that reflect your honest belief, and you would also see other stangers’ tweets. After seeing other people’s tweets, you would be asked about your belief about the Theory XYZ that claims that on a clear sunny

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
  0%|          | 0/3 [00:06<?, ?it/s]


KeyboardInterrupt: 